In [1]:
from biollm.task.perturbation.pert_task import PertTask


config_file = '../../config/pert/scgpt_pert.toml'
obj = PertTask(config_file)
pert_data, gene_ids = obj.make_dataset()

/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/lightning/fabric/plugins/environments/xla.py:18: DeprecationWarning: `ModuleAvailableCache` is a special case of `RequirementCache`. Please use `RequirementCache(module=...)` instead.
  from lightning.fabric.accelerators.tpu import _XLA_AVAILABLE, TPUAccelerator
/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/lightning/fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/share/huadjyin/home/s_qiuping1/app/miniconda3/envs/scgpt/lib/python3.9/site-packages/pkg_resources/__init__.py:2348: DeprecationWarning: Deprecated call 

Using simple batchnorm instead of domain specific batchnorm


Found local copy...


before filter: (86601, 4486)
match 1.0 genes in vocab of size 4486
after filter: View of AnnData object with n_obs × n_vars = 86601 × 4486
    obs: 'condition', 'cell_type', 'dose_val', 'control', 'condition_name', 'pert_gene_in_vocab'
    var: 'gene_name', 'is_in_vocab'
    uns: 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'top_non_zero_de_20'
    layers: 'counts'


Found local copy...
Local copy of pyg dataset is detected. Loading...
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['LYL1+IER5L' 'ctrl+IER5L' 'IER5L+ctrl']
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:10
combo_seen1:60
combo_seen2:13
unseen_single:36
Done!
Creating dataloaders....
Done!


here1


2024-04-19 10:43:25-pert_task[line-67]-INFO: match 4486/4486 genes in vocabulary of size 36574.


In [2]:
from biollm.evaluate.bm_metrices_pert import eval_scgpt_perturb


best_model_path = obj.args.save_dir + '/best_model.pt'
best_model = obj.load_obj.load_pretrain_model(best_model_path, obj.model)

In [ ]:
test_loader = pert_data.dataloader["test_loader"]
test_res = eval_scgpt_perturb(test_loader, best_model, 'cuda:2', 'all', gene_ids)

start to predict:  33479


In [11]:
from biollm.repo.gears.inference import evaluate, compute_metrics

test_metrics, test_pert_res = compute_metrics(test_res)

In [12]:
test_metrics

{'mse': 0.00273933126851121,
 'mse_de': 0.06093397819866358,
 'pearson': 0.9962483165861986,
 'pearson_de': 0.9846196783127752}

In [13]:
test_pert_res

{'ASCC3+ctrl': {'mse': 0.0029523405794619688,
  'pearson': 0.995882020706295,
  'mse_de': 0.03236165051044305,
  'pearson_de': 0.9902130785737836},
 'CHERP+ctrl': {'mse': 0.004281947712227697,
  'pearson': 0.9937532694870067,
  'mse_de': 0.024928976431482636,
  'pearson_de': 0.9916145841248042},
 'COPB1+ctrl': {'mse': 0.0032467751436858097,
  'pearson': 0.995789881017043,
  'mse_de': 0.00011161331093555967,
  'pearson_de': 0.9999765443582874},
 'DAD1+ctrl': {'mse': 0.0012100210985040885,
  'pearson': 0.9985385068885994,
  'mse_de': 0.009983436478645108,
  'pearson_de': 0.9985259687096666},
 'DDOST+ctrl': {'mse': 0.0016654822329111643,
  'pearson': 0.9982302182504258,
  'mse_de': 0.006313443727599816,
  'pearson_de': 0.9984089845533795},
 'DDRGK1+ctrl': {'mse': 0.0013774621163757548,
  'pearson': 0.9981656171441128,
  'mse_de': 0.01967601256974229,
  'pearson_de': 0.9975521367213223},
 'EIF2B2+ctrl': {'mse': 0.010324305337211962,
  'pearson': 0.9840836616931413,
  'mse_de': 0.5003412402

In [8]:
# test_metrics, test_pert_res = compute_metrics(test_res)
from biollm.evaluate.bm_metrices_pert import compute_perturbation_metrics
test_metrics = compute_perturbation_metrics(
    test_res, pert_data.adata[pert_data.adata.obs["condition"] == "ctrl"]
)
print(test_metrics)

{'pearson': 0.9962483165861986, 'pearson_de': 0.9846196783127752, 'pearson_delta': 0.8436601087393946, 'pearson_de_delta': 0.9297923819797913}


In [9]:
from biollm.repo.gears.inference import deeper_analysis, non_dropout_analysis
import numpy as np

deeper_res = deeper_analysis(pert_data.adata, test_res)
non_dropout_res = non_dropout_analysis(pert_data.adata, test_res)

metrics = ["pearson_delta", "pearson_delta_de"]
metrics_non_dropout = [
    "pearson_delta_top20_de_non_dropout",
    "pearson_top20_de_non_dropout",
]
subgroup_analysis = {}
for name in pert_data.subgroup["test_subgroup"].keys():
    subgroup_analysis[name] = {}
    for m in metrics:
        subgroup_analysis[name][m] = []

    for m in metrics_non_dropout:
        subgroup_analysis[name][m] = []

for name, pert_list in pert_data.subgroup["test_subgroup"].items():
    for pert in pert_list:
        for m in metrics:
            subgroup_analysis[name][m].append(deeper_res[pert][m])

        for m in metrics_non_dropout:
            subgroup_analysis[name][m].append(non_dropout_res[pert][m])

for name, result in subgroup_analysis.items():
    for m in result.keys():
        mean_value = np.mean(subgroup_analysis[name][m])
        print("test_" + name + "_" + m + ": " + str(mean_value))


test_combo_seen0_pearson_delta: nan
test_combo_seen0_pearson_delta_de: nan
test_combo_seen0_pearson_delta_top20_de_non_dropout: nan
test_combo_seen0_pearson_top20_de_non_dropout: nan
test_combo_seen1_pearson_delta: nan
test_combo_seen1_pearson_delta_de: nan
test_combo_seen1_pearson_delta_top20_de_non_dropout: nan
test_combo_seen1_pearson_top20_de_non_dropout: nan
test_combo_seen2_pearson_delta: nan
test_combo_seen2_pearson_delta_de: nan
test_combo_seen2_pearson_delta_top20_de_non_dropout: nan
test_combo_seen2_pearson_top20_de_non_dropout: nan
test_unseen_single_pearson_delta: 0.8436601087393947
test_unseen_single_pearson_delta_de: 0.9297923819797913
test_unseen_single_pearson_delta_top20_de_non_dropout: 0.9287463832873536
test_unseen_single_pearson_top20_de_non_dropout: 0.9840293158452901
